In [1]:
import pepdata

In [2]:
df = pepdata.iedb.mhc.load_dataframe()

In [6]:
ligand_mask = (df["Assay"]["Assay Group"] == "ligand presentation")

In [7]:
df_ligand = df[ligand_mask]

In [9]:
df_ligand["Assay"]["Method/Technique"].value_counts()

cellular MHC/mass spectrometry    447238
secreted MHC/mass spectrometry     34610
Edman degradation                    888
mass spectrometry                    624
coelution                            146
T cell recognition                   145
Name: Method/Technique, dtype: int64

In [14]:
class2_mask = df["MHC"]["MHC allele class"] == "II"

In [15]:
df_ligand_class2 = df[ligand_mask & class2_mask]

In [24]:
allele_mask = df_ligand_class2["MHC"]["Allele Name"].str.contains("\*")

In [25]:
allele_mask.sum()

26641

In [26]:
df_ligand_class2_4digit = df_ligand_class2[allele_mask]

In [36]:
ptm_mask = df_ligand_class2_4digit["Epitope"]["Description"].str.contains(" ")

In [37]:
df_ligand_class2_4digit_no_ptm = df_ligand_class2_4digit[~ptm_mask]

In [39]:
df_ligand_class2_4digit_no_ptm["MHC"]["Allele Name"].value_counts()

HLA-DQA1*05:05/DQB1*03:01    7431
HLA-DQA1*02:01/DQB1*02:02    7355
HLA-DQA1*05:01/DQB1*02:01    4399
HLA-DRB1*01:01               3638
HLA-DRB1*04:01               1100
HLA-DRB1*15:01                678
HLA-DRA*01:01/DRB1*08:01      486
HLA-DRA*01:01/DRB1*10:01      238
HLA-DRB5*01:01                223
HLA-DRB1*03:01                186
HLA-DRA*01:01/DRB1*04:01      154
HLA-DRB1*01:03                150
HLA-DRB1*11:01                104
HLA-DRB1*04:04                 55
HLA-DQA1*01:02/DQB1*06:02      53
HLA-DRB1*04:02                 51
HLA-DRB1*04:05                 35
HLA-DQA1*01:01/DQB1*05:01      33
HLA-DRB1*11:04                 24
HLA-DRB1*13:01                 24
HLA-DRB1*13:02                 22
HLA-DQA1*03:01/DQB1*03:02      20
HLA-DQA1*05:01/DQB1*03:02      19
HLA-DRB1*09:01                 18
BoLA-DRB3*1201                 18
HLA-DRB3*02:02                 13
HLA-DQA1*03:01/DQB1*02:01      13
HLA-DPA1*01:03/DPB1*02:01      12
BoLA-DRB3*2703                  6
HLA-DRB1*01:02

In [54]:
human_mask = df_ligand_class2_4digit_no_ptm["MHC"]["Allele Name"].str.startswith("HLA")

In [55]:
df_ligand_class2_4digit_no_ptm_human = df_ligand_class2_4digit_no_ptm[human_mask]

In [56]:
df_ligand_class2_4digit_no_ptm_human["MHC"]["Allele Name"].value_counts()

HLA-DQA1*05:05/DQB1*03:01    7431
HLA-DQA1*02:01/DQB1*02:02    7355
HLA-DQA1*05:01/DQB1*02:01    4399
HLA-DRB1*01:01               3638
HLA-DRB1*04:01               1100
HLA-DRB1*15:01                678
HLA-DRA*01:01/DRB1*08:01      486
HLA-DRA*01:01/DRB1*10:01      238
HLA-DRB5*01:01                223
HLA-DRB1*03:01                186
HLA-DRA*01:01/DRB1*04:01      154
HLA-DRB1*01:03                150
HLA-DRB1*11:01                104
HLA-DRB1*04:04                 55
HLA-DQA1*01:02/DQB1*06:02      53
HLA-DRB1*04:02                 51
HLA-DRB1*04:05                 35
HLA-DQA1*01:01/DQB1*05:01      33
HLA-DRB1*11:04                 24
HLA-DRB1*13:01                 24
HLA-DRB1*13:02                 22
HLA-DQA1*03:01/DQB1*03:02      20
HLA-DQA1*05:01/DQB1*03:02      19
HLA-DRB1*09:01                 18
HLA-DQA1*03:01/DQB1*02:01      13
HLA-DRB3*02:02                 13
HLA-DPA1*01:03/DPB1*02:01      12
HLA-DRB1*01:02                  6
HLA-DQB1*03:01                  6
HLA-DRB3*01:01

In [57]:
import mhcnames

In [92]:
from collections import defaultdict, OrderedDict

In [59]:
mhcnames.parse_allele_name("HLA-DQB1*03:01")

AlleleName(species='HLA', gene='DQB1', allele_family='03', allele_code='01')

In [60]:
mhcnames.class2.parse_classi_or_classii_allele_name("HLA-DQB1*03:01")

(AlleleName(species='HLA', gene='DQA1', allele_family='01', allele_code='02'),
 AlleleName(species='HLA', gene='DQB1', allele_family='03', allele_code='01'))

In [61]:
def expand_name(allele):
    parts = mhcnames.class2.parse_classi_or_classii_allele_name(allele)
    if len(parts) != 2:
        raise ValueError("Couldn't parse '%s'" % allele)
    a,b = parts
    return "%s-%s*%s:%s/%s*%s:%s" % (a.species, a.gene, a.allele_family, a.allele_code, b.gene, b.allele_family, b.allele_code)

In [106]:
df_result = pd.DataFrame({})
alleles = list(df_ligand_class2_4digit_no_ptm_human["MHC"]["Allele Name"])
df_result["inferred_allele"] = [expand_name(allele) for allele in alleles]
df_result["original_allele"] = alleles
df_result["peptide"] = list(df_ligand_class2_4digit_no_ptm_human["Epitope"]["Description"])
df_result["assay"] = list(df_ligand_class2_4digit_no_ptm_human["Assay"]["Method/Technique"])
df_result["cell_type"] = list(df_ligand_class2_4digit_no_ptm_human["Antigen Processing Cells"]["Cell Type"])
df_result["source_protein"] = list(df_ligand_class2_4digit_no_ptm_human["Epitope"]["Parent Protein"])
df_result["source_protein_species"] = list(df_ligand_class2_4digit_no_ptm_human["Epitope"]["Organism Name"])
df_result["reference_year"] = list(df_ligand_class2_4digit_no_ptm_human["Reference"]["Date"])
df_result["reference_title"] = list(df_ligand_class2_4digit_no_ptm_human["Reference"]["Title"])

In [107]:
df_result

,inferred_allele,original_allele,peptide,assay,cell_type,source_protein,source_protein_species,reference_year,reference_title
0,HLA-DRA1*01:01/DRB1*01:01,HLA-DRB1*01:01,AILEFRAMAQFSRKTD,Edman degradation,B cell,Nck-associated protein 1-like,Homo sapiens,1992,Self-peptide released from class II HLA-DR1 ex...
1,HLA-DRA1*01:01/DRB1*04:01,HLA-DRB1*04:01,VYPEVTVYPAKT,Edman degradation,B cell,"HLA class II histocompatibility antigen, DRB1-...",Homo sapiens,1995,Naturally processed peptides from rheumatoid a...
2,HLA-DRA1*01:01/DRB1*04:02,HLA-DRB1*04:02,MHHWLLFEMSRHSLE,Edman degradation,B cell,HLA class II histocompatibility antigen gamma ...,Homo sapiens,1995,Naturally processed peptides from rheumatoid a...
3,HLA-DRA1*01:01/DRB1*04:04,HLA-DRB1*04:04,ANIAVDKANLEIMTKR,Edman degradation,B cell,"HLA class II histocompatibility antigen, DR al...",Homo sapiens,1995,Naturally processed peptides from rheumatoid a...
4,HLA-DRA1*01:01/DRB1*15:01,HLA-DRB1*15:01,EAEQLRAYLDGTGVE,Edman degradation,B cell,NaN,Homo sapiens,1994,Ligand motifs of HLA-DRB5*0101 and DRB1*1501 m...
5,HLA-DRA1*01:01/DRB1*13:01,HLA-DRB1*13:01,TPKIQVYSRHPAENGKS,Edman degradation,B cell,Beta-2-microglobulin,Homo sapiens,1995,Naturally processed peptides from two disease-...
6,HLA-DRA1*01:01/DRB1*13:01,HLA-DRB1*13:01,TPKIQVYSRHPAENGK,Edman degradation,B cell,Beta-2-microglobulin,Homo sapiens,1995,Naturally processed peptides from two disease-...
7,HLA-DRA1*01:01/DRB1*13:01,HLA-DRB1*13:01,TPKIQVYSRHPAENG,Edman degradation,B cell,Beta-2-microglobulin,Homo sapiens,1995,Naturally processed peptides from two disease-...
8,HLA-DRA1*01:01/DRB1*13:01,HLA-DRB1*13:01,TPKIQVYSRHPAEN,Edman degradation,B cell,Beta-2-microglobulin,Homo sapiens,1995,Naturally processed peptides from two disease-...
9,HLA-DRA1*01:01/DRB1*13:01,HLA-DRB1*13:01,TERVRLVTRHIYNREE,Edman degradation,B cell,"HLA class II histocompatibility antigen, DQ be...",Homo sapiens,1995,Naturally processed peptides from two disease-...


In [102]:
df_result.allele.value_counts()

HLA-DQA1*05:05/DQB1*03:01    7431
HLA-DQA1*02:01/DQB1*02:02    7355
HLA-DQA1*05:01/DQB1*02:01    4399
HLA-DRA1*01:01/DRB1*01:01    3638
HLA-DRA1*01:01/DRB1*04:01    1254
HLA-DRA1*01:01/DRB1*15:01     678
HLA-DRA1*01:01/DRB1*08:01     486
HLA-DRA1*01:01/DRB1*10:01     238
HLA-DRA1*01:01/DRB5*01:01     223
HLA-DRA1*01:01/DRB1*03:01     186
HLA-DRA1*01:01/DRB1*01:03     150
HLA-DRA1*01:01/DRB1*11:01     104
HLA-DQA1*01:02/DQB1*06:02      55
HLA-DRA1*01:01/DRB1*04:04      55
HLA-DRA1*01:01/DRB1*04:02      51
HLA-DRA1*01:01/DRB1*04:05      35
HLA-DQA1*01:01/DQB1*05:01      33
HLA-DRA1*01:01/DRB1*13:01      24
HLA-DRA1*01:01/DRB1*11:04      24
HLA-DRA1*01:01/DRB1*13:02      22
HLA-DQA1*03:01/DQB1*03:02      20
HLA-DQA1*05:01/DQB1*03:02      19
HLA-DRA1*01:01/DRB1*09:01      18
HLA-DQA1*03:01/DQB1*02:01      13
HLA-DRA1*01:01/DRB3*02:02      13
HLA-DPA1*01:03/DPB1*02:01      12
HLA-DRA1*01:01/DRB1*01:02       6
HLA-DQA1*01:02/DQB1*03:01       6
HLA-DPA1*01:03/DPB1*04:01       4
HLA-DRA1*01:01

In [103]:
df_result.peptide.str.len().value_counts()

15    4363
14    4348
16    3769
13    3157
17    2661
12    1777
18    1555
11     970
19     927
20     628
10     596
9      573
21     409
22     241
23     169
24     121
8       89
25      82
27      20
26      15
28      15
29      12
31      11
30       9
32       9
33       5
37       4
43       4
36       4
7        4
34       4
35       3
38       2
6        1
39       1
3        1
46       1
5        1
Name: peptide, dtype: int64

In [104]:
df_result.assay.value_counts()

cellular MHC/mass spectrometry    25998
Edman degradation                   297
secreted MHC/mass spectrometry      257
coelution                             9
Name: assay, dtype: int64

In [109]:
df_result.to_csv("iedb-mhc2-ligands-7-29-2018.csv", index=False)

In [112]:
len(df_result["inferred_allele"].unique())

33

In [113]:
df_result.assay.value_counts()

cellular MHC/mass spectrometry    25998
Edman degradation                   297
secreted MHC/mass spectrometry      257
coelution                             9
Name: assay, dtype: int64